In [1]:
import pandas as pd
import os
from dotenv import load_dotenv

In [2]:
HOME = "/home/aircsrv5/Quan/DataOps/DataOps-bootstrap/demo"
load_dotenv(dotenv_path=f'{HOME}/.env')

DATA_DIR = os.environ.get('DATA_PATH')

In [2]:
# DATA_PATH = f'{DATA_DIR}/small_canada.csv'
import pandas as pd
df = pd.read_csv('/home/aircsrv5/Quan/DataOps/DataOps-bootstrap/demo/include/data/cleaned_canada.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44896 entries, 0 to 44895
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   City            44896 non-null  object 
 1   Province        44896 non-null  object 
 2   Latitude        44896 non-null  float64
 3   Longitude       44896 non-null  float64
 4   Price           44896 non-null  float64
 5   Bedrooms        44896 non-null  float64
 6   Bathrooms       44896 non-null  float64
 7   Acreage         44896 non-null  float64
 8   Property Type   44896 non-null  object 
 9   Square Footage  44896 non-null  float64
 10  Garage          44896 non-null  object 
 11  Parking         44896 non-null  object 
 12  Basement        14962 non-null  object 
 13  Exterior        17439 non-null  object 
 14  Fireplace       44896 non-null  object 
 15  Heating         38242 non-null  object 
 16  Flooring        15216 non-null  object 
 17  Roof            10052 non-null 

In [4]:
import great_expectations as gx
import great_expectations.expectations as gxe

# Markdown

## Define GX context

In [5]:
# context = gx.get_context(mode="file")
# Retrieve your Data Context
context = gx.get_context()
assert type(context).__name__ == "EphemeralDataContext"

## Define Data Source

In [6]:
import great_expectations as gx

# Define the Data Source name
data_source_name = "canada_data_source"

# Add the Data Source to the Data Context
data_source = context.data_sources.add_pandas(name=data_source_name)
assert data_source.name == data_source_name

## Define Data Asset

In [7]:
import great_expectations as gx

# Retrieve the Data Source
data_source_name = "canada_data_source"
data_source = context.data_sources.get(data_source_name)

# Define the Data Asset name
data_asset_name = "canada_data_asset"

# Add a Data Asset to the Data Source
data_asset = data_source.add_dataframe_asset(name=data_asset_name)

assert data_asset.name == data_asset_name

## Define a Batch Definition

In [8]:
import great_expectations as gx

# Retrieve the Data Asset
data_source_name = "canada_data_source"
data_asset_name = "canada_data_asset"
data_asset = context.data_sources.get(data_source_name).get_asset(data_asset_name)

# Define the Batch Definition name
batch_definition_name = "canada_batch_definition"

# Add a Batch Definition to the Data Asset
batch_definition = data_asset.add_batch_definition_whole_dataframe(
    batch_definition_name
)
assert batch_definition.name == batch_definition_name

## Define a Batch Parameters

In [9]:
import pandas as pd

DATA_PATH = f'{DATA_DIR}/small_canada.csv'

dataframe = pd.read_csv('/home/aircsrv5/Quan/DataOps/DataOps-bootstrap/demo/include/data/yellow_tripdata_sample_2019-01.csv')
dataframe.head()

batch_parameters = {"dataframe": dataframe}

## Define Expectations and Validate Batch against expectations

In [10]:
import great_expectations as gx

# Retrieve the dataframe Batch Definition
data_source_name = "canada_data_source"
data_asset_name = "canada_data_asset"
batch_definition_name = "canada_batch_definition"
batch_definition = (
    context.data_sources.get(data_source_name)
    .get_asset(data_asset_name)
    .get_batch_definition(batch_definition_name)
)

# Create an Expectation to test
expectation = gx.expectations.ExpectColumnValuesToBeBetween(
    column="passenger_count", max_value=6, min_value=1
)

# Get the dataframe as a Batch
batch = batch_definition.get_batch(batch_parameters=batch_parameters)

# Test the Expectation
validation_results = batch.validate(expectation)
print(validation_results)

Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 2805.93it/s]

{
  "success": true,
  "expectation_config": {
    "type": "expect_column_values_to_be_between",
    "kwargs": {
      "batch_id": "canada_data_source-canada_data_asset",
      "column": "passenger_count",
      "min_value": 1.0,
      "max_value": 6.0
    },
    "meta": {}
  },
  "result": {
    "element_count": 10000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_counts": [],
    "partial_unexpected_index_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


In [ ]:
context.build_data_docs()


In [13]:
context.open_data_docs()

In [1]:
import pandas as pd

data_ab = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_ab.csv', low_memory=False)
data_bc = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_bc.csv', low_memory=False)
data_mb = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_mb.csv', low_memory=False)
data_nb = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_nb.csv', low_memory=False)
data_nl = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_nl.csv', low_memory=False)
data_ns = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_ns.csv', low_memory=False)
data_nt = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_nt.csv', low_memory=False)
data_on = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_on.csv', low_memory=False)
data_pe = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_pe.csv', low_memory=False)
data_sk = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_sk.csv', low_memory=False)
data_yt = pd.read_csv('/home/aircsrv5/Quan/DataOps/Data-for-testing/canada-house-price/archive-2/data_yt.csv', low_memory=False)

In [2]:
df = pd.concat([data_ab, data_bc, data_mb, data_nb, data_nl, 
                data_ns, data_nt, data_on, data_pe, data_sk,
                data_yt], axis=0) 
df.shape



(133725, 396)

In [3]:
df.to_csv("full_canada.csv", index=False)

In [6]:
file_path = "/home/aircsrv5/Quan/DataOps/DataOps-bootstrap/full_canada.csv"
df = pd.read_csv(file_path)

/tmp/ipykernel_2444356/274533351.py:2: DtypeWarning: Columns (42,87,93,259) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [7]:
with open(file_path) as f:
    for i, line in enumerate(f):
        if i == 11091:  # zero-indexed, so line 11092 is at index 11091
            print(line)
            break

Upstairs, the primary retreat impresses with a massive walk-in closet and a spa-like ensuite featuring dual vanities, a tiled soaker tub, a stand-up shower, and two large windows. Down the hall, you’ll find two additional bedrooms, a shared 4-piece bath, an upper-floor laundry with a side-by-side washer and dryer, and a bright bonus room for family relaxation.

